**Pomembni stolpci**

In [8]:
import pandas as pd

df = pd.read_csv('sample.csv', sep=',')

pomembni = df[['smart_5_raw','smart_187_raw','smart_188_raw']]
display(pomembni)
display(pomembni.isnull().sum())

,smart_5_raw,smart_187_raw,smart_188_raw
0,0.0,0.0,NaN
1,0.0,0.0,NaN
2,0.0,0.0,NaN
3,0.0,0.0,NaN
4,0.0,0.0,NaN
...,...,...,...
329482,0.0,NaN,NaN
329483,0.0,NaN,NaN
329484,0.0,NaN,NaN
329485,0.0,NaN,NaN


smart_5_raw        2264
smart_187_raw    216540
smart_188_raw    217083
dtype: int64

**Rough Preprocessing**

In [9]:
import pandas as pd

df = pd.read_csv('sample.csv', sep=',')
display(df.head(5))
display(df.shape)

NaN = df.isnull().mean()

nujni_stolpci = [
    'smart_5_raw', 
    'smart_187_raw', 
    'smart_188_raw', 
    'smart_197_raw', 
    'smart_198_raw', 
    'smart_9_raw', 
    'smart_194_raw', 
    'capacity_bytes', 
    'model',
    'failure'
]

#posodobi logiko izbire relevantnih stolpcev
#obdrži stolpec, če ima < 30% NaN ALI če je na seznamu nujnih
relevantni = NaN[(NaN < 0.3) | (NaN.index.isin(nujni_stolpci))].index.tolist()

print(f"Obdržali bomo {len(relevantni)} stolpcev.")

zaStran = [col for col in df.columns if col not in relevantni]
df = df.drop(columns=zaStran)

#POMEMBNO - Zapolni NaN v teh nujnih stolpcih z 0
# Ker XGBoost ne mara popolnih praznin, manjše število NaN pa zna rešiti, 
df[nujni_stolpci] = df[nujni_stolpci].fillna(0)
display(df.shape)
display(df)

# "lokacijski stolpci" od blackblaze-a, ki jih ne rabimo
lokacijski = df[['datacenter','cluster_id','vault_id','pod_id','pod_slot_num','is_legacy_format']]

# vsi "normalized" so samo interpetacija proizvajalca, ker je vec proizvajalcev so nekonsistentni
def vsiNorm(df):
    normStolpci = []

    for col in df.columns:
        if 'normalized' in col:
            normStolpci.append(col)
    return normStolpci

normStolpci = vsiNorm(df)
df = df.drop(columns=normStolpci)
df = df.drop(columns=lokacijski)
display(df.shape)   #21 dejansko uporabnih stolpcev ..
display(df.columns)

#df.to_csv('izbolsani_podatki.csv', index=False)

,date,serial_number,model,capacity_bytes,failure,datacenter,cluster_id,vault_id,pod_id,pod_slot_num,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,2025-09-29,2206E608DB42,CT250MX500SSD1,250059350016,0,sac0,0,1028,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-09-29,2207E60CC65A,CT250MX500SSD1,250059350016,0,sac0,0,1028,13,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-09-29,2340E87B92B5,CT250MX500SSD1,250059350016,0,sac0,0,1028,14,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-09-29,2340E87B97E8,CT250MX500SSD1,250059350016,0,sac0,0,1028,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-09-29,2407E896B6D5,CT250MX500SSD1,250059350016,0,sac0,0,1028,8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(329487, 197)

Obdržali bomo 41 stolpcev.


(329487, 41)

,date,serial_number,model,capacity_bytes,failure,datacenter,cluster_id,vault_id,pod_id,pod_slot_num,...,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw
0,2025-09-29,2206E608DB42,CT250MX500SSD1,250059350016,0,sac0,0,1028,4,NaN,...,NaN,NaN,70.0,30.0,100.0,0.0,100.0,0.0,100.0,0.0
1,2025-09-29,2207E60CC65A,CT250MX500SSD1,250059350016,0,sac0,0,1028,13,NaN,...,NaN,NaN,65.0,35.0,100.0,0.0,100.0,0.0,100.0,0.0
2,2025-09-29,2340E87B92B5,CT250MX500SSD1,250059350016,0,sac0,0,1028,14,NaN,...,NaN,NaN,67.0,33.0,100.0,0.0,100.0,0.0,100.0,0.0
3,2025-09-29,2340E87B97E8,CT250MX500SSD1,250059350016,0,sac0,0,1028,2,NaN,...,NaN,NaN,64.0,36.0,100.0,0.0,100.0,0.0,100.0,0.0
4,2025-09-29,2407E896B6D5,CT250MX500SSD1,250059350016,0,sac0,0,1028,8,NaN,...,NaN,NaN,72.0,28.0,100.0,0.0,100.0,0.0,100.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329482,2025-09-29,3ZHG6U1Z,WDC WUH722222ALE6L4,22000969973760,0,yyz1,60,1002,4,48.0,...,100.0,328.0,64.0,32.0,100.0,0.0,100.0,0.0,100.0,0.0
329483,2025-09-29,3ZHG6U9Z,WDC WUH722222ALE6L4,22000969973760,0,yyz1,60,1002,2,24.0,...,100.0,327.0,71.0,26.0,100.0,0.0,100.0,0.0,100.0,0.0
329484,2025-09-29,ZGG6LV5B,WDC WUH722222ALE6L4,22000969973760,0,yyz1,60,1002,11,8.0,...,100.0,327.0,64.0,32.0,100.0,0.0,100.0,0.0,100.0,0.0
329485,2025-09-29,ZGG7J1TA,WDC WUH722222ALE6L4,22000969973760,0,yyz1,60,1002,7,0.0,...,100.0,330.0,67.0,29.0,100.0,0.0,100.0,0.0,100.0,0.0


(329487, 21)

Index(['date', 'serial_number', 'model', 'capacity_bytes', 'failure',
       'smart_1_raw', 'smart_3_raw', 'smart_4_raw', 'smart_5_raw',
       'smart_7_raw', 'smart_9_raw', 'smart_10_raw', 'smart_12_raw',
       'smart_187_raw', 'smart_188_raw', 'smart_192_raw', 'smart_193_raw',
       'smart_194_raw', 'smart_197_raw', 'smart_198_raw', 'smart_199_raw'],
      dtype='object')

**Vrte diskov**

In [19]:
unikatneVrsteDiskov = df['model'].unique()
print("vrste diskov ", unikatneVrsteDiskov)

#ssd ali hdd
ssd_keywords = ['SSD', 'MTFD', 'SSDSC', '850 PRO', '870 EVO', '860 PRO', '5300']
df['jeSSD'] = df['model'].apply(lambda x: 1 if any(k in str(x).upper() for k in ssd_keywords) else 0)

#model
def getModel(model):
    m = str(model).upper()
    if m.startswith('ST') or 'SEAGATE' in m: return 'Seagate'
    if m.startswith('WDC') or m.startswith('WD') or 'WESTERN' in m: return 'Western Digital'
    if m.startswith('HGST') or m.startswith('HUH') or m.startswith('HMS'): return 'HGST'
    if m.startswith('TOSHIBA') or m.startswith('MG'): return 'Toshiba'
    if m.startswith('SAMSUNG'): return 'Samsung'
    if m.startswith('CT') or 'CRUCIAL' in m: return 'Crucial'
    return 'Other'

df['model'] = df['model'].apply(getModel)

display(df)
ostali = df[df['model'] == 'Other']['model'].unique()
print("nekategorizirani modeli: ", ostali)
modeli = df['model'].unique()
display("kategorizirani modeli: ", modeli)

vrste diskov  ['Crucial' 'HGST' 'Seagate' 'Toshiba' 'Western Digital' 'Other' 'Samsung']


,date,serial_number,model,capacity_bytes,failure,smart_1_raw,smart_3_raw,smart_4_raw,smart_5_raw,smart_7_raw,...,smart_12_raw,smart_187_raw,smart_188_raw,smart_192_raw,smart_193_raw,smart_194_raw,smart_197_raw,smart_198_raw,smart_199_raw,jeSSD
0,2025-09-29,2206E608DB42,Crucial,250059350016,0,0.0,NaN,NaN,0.0,NaN,...,15.0,0.0,0.0,NaN,NaN,30.0,0.0,0.0,0.0,0
1,2025-09-29,2207E60CC65A,Crucial,250059350016,0,0.0,NaN,NaN,0.0,NaN,...,3.0,0.0,0.0,NaN,NaN,35.0,0.0,0.0,0.0,0
2,2025-09-29,2340E87B92B5,Crucial,250059350016,0,0.0,NaN,NaN,0.0,NaN,...,6.0,0.0,0.0,NaN,NaN,33.0,0.0,0.0,0.0,0
3,2025-09-29,2340E87B97E8,Crucial,250059350016,0,0.0,NaN,NaN,0.0,NaN,...,2.0,0.0,0.0,NaN,NaN,36.0,0.0,0.0,0.0,0
4,2025-09-29,2407E896B6D5,Crucial,250059350016,0,0.0,NaN,NaN,0.0,NaN,...,13.0,0.0,0.0,NaN,NaN,28.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329482,2025-09-29,3ZHG6U1Z,Western Digital,22000969973760,0,0.0,275.0,9.0,0.0,0.0,...,9.0,0.0,0.0,328.0,328.0,32.0,0.0,0.0,0.0,0
329483,2025-09-29,3ZHG6U9Z,Western Digital,22000969973760,0,0.0,240.0,9.0,0.0,0.0,...,9.0,0.0,0.0,327.0,327.0,26.0,0.0,0.0,0.0,0
329484,2025-09-29,ZGG6LV5B,Western Digital,22000969973760,0,0.0,267.0,9.0,0.0,0.0,...,9.0,0.0,0.0,327.0,327.0,32.0,0.0,0.0,0.0,0
329485,2025-09-29,ZGG7J1TA,Western Digital,22000969973760,0,0.0,337.0,10.0,0.0,0.0,...,10.0,0.0,0.0,330.0,330.0,29.0,0.0,0.0,0.0,0


nekategorizirani modeli:  ['Other']


'kategorizirani modeli: '

array(['Crucial', 'HGST', 'Seagate', 'Toshiba', 'Western Digital',
       'Other', 'Samsung'], dtype=object)